In [32]:
import os, glob
import pandas as pd
import warnings
warnings.filterwarnings('ignore')

In [33]:
from turtle import width


PATH = r"C:\Users\Admin\Desktop\Heig-Annee3\Semestre1\GML\Projet\crapauduc\test_cam1\outputs" # Path to the folder containing the images
bboxes = [] # List to store all bounding boxes
height = 0
width = 0
class_name = ""


for file in glob.glob(os.path.join(PATH, "*.xml")):
    with open(file, "r") as f:
        lines = f.readlines()
        bbox = []
        bbox_h_w_name = []
        filename = file.split("\\")[-1].split(".")[0]
        for line in lines:
            if "<xmin>" in line:
                bbox.append(int(line.split(">")[1].split("<")[0]))
            if "<ymin>" in line:
                bbox.append(int(line.split(">")[1].split("<")[0]))
            if "<xmax>" in line:
                bbox.append(int(line.split(">")[1].split("<")[0]))
            if "<ymax>" in line:
                bbox.append(int(line.split(">")[1].split("<")[0]))
            if "<width>" in line:
                width = int(line.split(">")[1].split("<")[0])
            if "<height>" in line:
                height = int(line.split(">")[1].split("<")[0])
            if "<name>" in line:
                class_name = line.split(">")[1].split("<")[0]

            bbox_h_w_name = [bbox, width, height, class_name, filename]
        bboxes.append(bbox_h_w_name)
            

In [34]:
bboxes

[[[295, 460, 728, 719],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_07'],
 [[391, 412, 740, 629],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_08'],
 [[409, 388, 770, 677],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_09'],
 [[487, 388, 872, 671],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_10'],
 [[566, 376, 896, 599],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_11'],
 [[590, 352, 932, 599],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_12'],
 [[674, 346, 1010, 605],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_13'],
 [[698, 346, 1010, 587],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_14'],
 [[704, 334, 1004, 605],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_15'],
 [[710, 352, 998, 611],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_16'],
 [[698, 358, 1016, 599],
  1920,
  1080,
  'grenouille-crapaud',
  'T04_03_2017_21_34_1

In [35]:
def xml_to_yolo_bbox(bbox, w, h):
    # xmin, ymin, xmax, ymax
    x_center = ((bbox[2] + bbox[0]) / 2) / w
    y_center = ((bbox[3] + bbox[1]) / 2) / h
    width = (bbox[2] - bbox[0]) / w
    height = (bbox[3] - bbox[1]) / h
    return [x_center, y_center, width, height]

In [36]:
columns = ["class", "top_left_x", "top_left_y", "w", "h", "img_size_x", "img_size_y", "name"]
df = pd.DataFrame(columns=columns)

for bbox in bboxes:
    
    yolo_bbox = xml_to_yolo_bbox(bbox[0], bbox[1], bbox[2])
    row = [bbox[3], yolo_bbox[0], yolo_bbox[1], yolo_bbox[2], yolo_bbox[3], bbox[1], bbox[2], bbox[4]]
    
    df = df.append(pd.Series(row, index=columns), ignore_index=True)


In [37]:
df

,class,top_left_x,top_left_y,w,h,img_size_x,img_size_y,name
0,grenouille-crapaud,0.266406,0.545833,0.225521,0.239815,1920,1080,T04_03_2017_21_34_07
1,grenouille-crapaud,0.294531,0.481944,0.181771,0.200926,1920,1080,T04_03_2017_21_34_08
2,grenouille-crapaud,0.307031,0.493056,0.188021,0.267593,1920,1080,T04_03_2017_21_34_09
3,grenouille-crapaud,0.353906,0.490278,0.200521,0.262037,1920,1080,T04_03_2017_21_34_10
4,grenouille-crapaud,0.380729,0.451389,0.171875,0.206481,1920,1080,T04_03_2017_21_34_11
...,...,...,...,...,...,...,...,...
181,triton,0.466667,0.317593,0.06875,0.172222,1920,1080,T27_03_2017_06_53_55
182,triton,0.471354,0.32037,0.078125,0.188889,1920,1080,T27_03_2017_06_53_56
183,triton,0.465885,0.313426,0.042188,0.197222,1920,1080,T27_03_2017_06_53_57
184,triton,0.461979,0.306481,0.046875,0.194444,1920,1080,T27_03_2017_06_53_58


In [38]:
df.to_csv("bounding_boxes_test.csv", index=False)